In [26]:
from keras.layers import Input, Dense
from keras.models import Model

In [27]:
from keras.datasets import mnist
import numpy as np

In [28]:
(x_train, _), (x_test, _) = mnist.load_data()

In [29]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [30]:
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

In [31]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.2499 - val_loss: 0.1708
Epoch 2/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.1514 - val_loss: 0.1359
Epoch 3/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.1298 - val_loss: 0.1221
Epoch 4/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.1193 - val_loss: 0.1149
Epoch 5/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.1129 - val_loss: 0.1092
Epoch 6/100
43264/60000 [====================>.........] - ETA: 0s - loss: 0.1089

KeyboardInterrupt: 

In [25]:
autoencoder